# Contextual Word Sentiment Classification
#### By Wael Ben Slima and Marko Babic


In [28]:
import pandas as pd    # to load dataset
import numpy as np     # for mathematic equation
from nltk.corpus import stopwords   # to get collection of stopwords
from sklearn.model_selection import train_test_split       # for splitting dataset
from tensorflow.keras.preprocessing.text import Tokenizer  # to encode text to int
from tensorflow.keras.preprocessing.sequence import pad_sequences   # to do padding or truncating
from tensorflow.keras.models import Sequential     # the model
from tensorflow.keras.layers import Embedding, LSTM, Dense # layers of the architecture
from tensorflow.keras.callbacks import ModelCheckpoint   # save model
from tensorflow.keras.models import load_model   # load saved model
import re

In [30]:
data = pd.read_csv('IMDB Dataset.csv')

print(data)

                                                  review sentiment
0      One of the other reviewers has mentioned that ...  positive
1      A wonderful little production. <br /><br />The...  positive
2      I thought this was a wonderful way to spend ti...  positive
3      Basically there's a family where a little boy ...  negative
4      Petter Mattei's "Love in the Time of Money" is...  positive
...                                                  ...       ...
49995  I thought this movie did a down right good job...  positive
49996  Bad plot, bad dialogue, bad acting, idiotic di...  negative
49997  I am a Catholic taught in parochial elementary...  negative
49998  I'm going to have to disagree with the previou...  negative
49999  No one expects the Star Trek movies to be high...  negative

[50000 rows x 2 columns]



<b>Stop Word</b> is a commonly used words in a sentence, usually a search engine is programmed to ignore this words (i.e. "the", "a", "an", "of", etc.)

<i>Declaring the english stop words</i>

In [33]:
english_stops = set(stopwords.words('english'))

<hr>

### TASK 1Load and Clean Dataset

In the original dataset, the reviews are still dirty. There are still html tags, numbers, uppercase, and punctuations. This will not be good for training, so in <b>load_dataset()</b> function, beside loading the dataset using <b>pandas</b>, I also pre-process the reviews by removing html tags, non alphabet (punctuations and numbers), stop words, and lower case all of the reviews.

### Encode Sentiments
In the same function, I also encode the sentiments into integers (0 and 1). Where 0 is for negative sentiments and 1 is for positive sentiments.

In [36]:
def load_dataset():
    df = pd.read_csv('IMDB Dataset.csv')
    x_data = df['review']       # Reviews/Input
    y_data = df['sentiment']    # Sentiment/Output

    # PRE-PROCESS REVIEW
    x_data = x_data.replace({'<.*?>': ''}, regex = True)          # remove html tag
    x_data = x_data.replace({'[^A-Za-z]': ' '}, regex = True)     # remove non alphabet
    x_data = x_data.apply(lambda review: [w for w in review.split() if w not in english_stops])  # remove stop words
    x_data = x_data.apply(lambda review: [w.lower() for w in review])   # lower case
    
    # ENCODE SENTIMENT -> 0 & 1
    y_data = y_data.replace('positive', 1)
    y_data = y_data.replace('negative', 0)

    return x_data, y_data

x_data, y_data = load_dataset()

print('Reviews')
print(x_data, '\n')
print('Sentiment')
print(y_data)

Reviews
0        [one, reviewers, mentioned, watching, oz, epis...
1        [a, wonderful, little, production, the, filmin...
2        [i, thought, wonderful, way, spend, time, hot,...
3        [basically, family, little, boy, jake, thinks,...
4        [petter, mattei, love, time, money, visually, ...
                               ...                        
49995    [i, thought, movie, right, good, job, it, crea...
49996    [bad, plot, bad, dialogue, bad, acting, idioti...
49997    [i, catholic, taught, parochial, elementary, s...
49998    [i, going, disagree, previous, comment, side, ...
49999    [no, one, expects, star, trek, movies, high, a...
Name: review, Length: 50000, dtype: object 

Sentiment
0        1
1        1
2        1
3        0
4        1
        ..
49995    1
49996    0
49997    0
49998    0
49999    0
Name: sentiment, Length: 50000, dtype: int64


C:\Users\MB\AppData\Local\Temp\ipykernel_14372\1137108278.py:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_data = y_data.replace('negative', 0)


### Split Dataset
In this work, I decided to split the data into 80% of Training and 20% of Testing set using <b>train_test_split</b> method from Scikit-Learn. By using this method, it automatically shuffles the dataset. We need to shuffle the data because in the original dataset, the reviews and sentiments are in order, where they list positive reviews first and then negative reviews. By shuffling the data, it will be distributed equally in the model, so it will be more accurate for predictions.

In [38]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.2)

print('Train Set')
print(x_train, '\n')
print(x_test, '\n')
print('Test Set')
print(y_train, '\n')
print(y_test)

Train Set
22842    [ty, cobb, far, interesting, belligerently, in...
25265    [i, originally, gave, episode, rating, two, i,...
15386    [there, many, different, versions, one, floati...
6706     [there, really, one, thing, say, sorry, movie,...
15551    [this, another, one, movies, could, great, the...
                               ...                        
22546    [my, wife, i, rented, movie, people, drawn, pa...
19941    [this, movie, amazing, i, tears, end, movie, o...
1051     [i, rented, movie, friend, good, laugh, we, ac...
2156     [a, humorous, voyage, normally, somber, funera...
28248    [it, really, cheesy, parody, tomb, raider, ind...
Name: review, Length: 40000, dtype: object 

5944     [excellent, film, suzy, kendall, hold, interes...
35724    [i, agree, jerry, it, underrated, space, movie...
11089    [i, found, film, one, great, heart, warming, g...
9832     [i, rented, film, yesterday, mostly, due, good...
44598    [not, one, films, one, biggest, pieces, tripe,...
 

<hr>
<i>Function for getting the maximum review length, by calculating the mean of all the reviews length (using <b>numpy.mean</b>)</i>

In [40]:
def get_max_length():
    review_length = []
    for review in x_train:
        review_length.append(len(review))

    return int(np.ceil(np.mean(review_length)))

<hr>

### TASK 2 : Tokenize and Pad/Truncate Reviews
A Neural Network only accepts numeric data, so we need to encode the reviews. I use <b>tensorflow.keras.preprocessing.text.Tokenizer</b> to encode the reviews into integers, where each unique word is automatically indexed (using <b>fit_on_texts</b> method) based on <b>x_train</b>. <br>
<b>x_train</b> and <b>x_test</b> is converted into integers using <b>texts_to_sequences</b> method.

Each reviews has a different length, so we need to add padding (by adding 0) or truncating the words to the same length (in this case, it is the mean of all reviews length) using <b>tensorflow.keras.preprocessing.sequence.pad_sequences</b>.


<b>post</b>, pad or truncate the words in the back of a sentence<br>
<b>pre</b>, pad or truncate the words in front of a sentence

In [42]:
# ENCODE REVIEW
token = Tokenizer(lower=False)    # no need lower, because already lowered the data in load_data()
token.fit_on_texts(x_train)
x_train = token.texts_to_sequences(x_train)
x_test = token.texts_to_sequences(x_test)

max_length = get_max_length()

x_train = pad_sequences(x_train, maxlen=max_length, padding='post', truncating='post')
x_test = pad_sequences(x_test, maxlen=max_length, padding='post', truncating='post')

total_words = len(token.word_index) + 1   # add 1 because of 0 padding

print('Encoded X Train\n', x_train, '\n')
print('Encoded X Test\n', x_test, '\n')
print('Maximum review length: ', max_length)

Encoded X Train
 [[12613  6826   132 ...    42    14   893]
 [    1  1763   407 ...    76     6  1116]
 [   49    38   182 ...   813  4779   894]
 ...
 [    1  1514     3 ...     0     0     0]
 [   39  1902  5151 ...     0     0     0]
 [    7    13   854 ...     0     0     0]] 

Encoded X Test
 [[  224     4 10946 ...     0     0     0]
 [    1   926  1555 ...     0     0     0]
 [    1   161     4 ...     0     0     0]
 ...
 [ 3033  5999  1326 ...     0     0     0]
 [   20   115    21 ...     0     0     0]
 [    8     3  1147 ... 18922    10   280]] 

Maximum review length:  130


### TASK 3 Build Architecture/Model

In [48]:
import requests
import io
import zipfile
import numpy as np
from torch.utils.data import Dataset
import pandas as pd

# Step 1: Load GloVe embeddings into a dictionary
def load_glove_embeddings(url, embedding_dim=50):
    response = requests.get(url)
    
    if response.status_code == 200:
        with zipfile.ZipFile(io.BytesIO(response.content)) as zip_ref:
            with zip_ref.open(f'glove.6B.{embedding_dim}d.txt') as f:
                embedding_model = {}
                for line in f:
                    values = line.decode('utf-8').split()
                    word = values[0]
                    embedding_vector = np.array(values[1:], dtype='float32')
                    embedding_model[word] = embedding_vector
                print("GloVe embeddings loaded successfully!")
                return embedding_model
    else:
        print(f"Failed to download the GloVe file. Status code: {response.status_code}")
        return None

# Step 2: Define the dataset class
class WordContextDataset(Dataset):
    def __init__(self, df, word_scores, embedding_model, window_size=2):
        """
        Args:
        - df: A pandas DataFrame containing the text data.
        - word_scores: A dictionary containing words and their corresponding scores.
        - embedding_model: A dictionary containing word embeddings (e.g., GloVe).
        - window_size: Size of the context window around the target word.
        """
        if embedding_model is None:
            raise ValueError("Embedding model is None. Please check if the GloVe embeddings were loaded correctly.")
        
        self.texts = df['text'].tolist()
        self.word_scores = word_scores
        self.embedding_model = embedding_model
        self.window_size = window_size
        self.embedding_dim = next(iter(self.embedding_model.values())).shape[0]
        self.data = self._prepare_dataset()

    def _prepare_dataset(self):
        data = []
        for text in self.texts:
            tokens = text.split()
            for i, target_word in enumerate(tokens):
                if target_word not in self.embedding_model:
                    continue
                
                # Get context window
                start = max(i - self.window_size, 0)
                end = min(i + self.window_size + 1, len(tokens))
                context_words = tokens[start:i] + tokens[i+1:end]

                # Compute embeddings
                target_embedding = self.embedding_model.get(target_word, np.zeros(self.embedding_dim))
                context_embeddings = [
                    self.embedding_model.get(word, np.zeros(self.embedding_dim)) 
                    for word in context_words
                ]
                if context_embeddings:
                    averaged_context_embedding = np.mean(context_embeddings, axis=0)
                else:
                    averaged_context_embedding = np.zeros(self.embedding_dim)
                
                # Add to dataset
                score = self.word_scores.get(target_word, 0)
                data.append({
                    'target_word': target_word,
                    'target_embedding': target_embedding,
                    'context_embedding': averaged_context_embedding,
                    'score': score
                })
        return data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# Example usage
if __name__ == "__main__":
    # Corrected URL
    url = "https://nlp.stanford.edu/data/glove.6B.zip"

    # Load GloVe embeddings
    embedding_model = load_glove_embeddings(url, 50)
    
    # Check if embeddings were loaded
    if embedding_model is not None:
        # Example DataFrame
        data = {
            "text": [
                "the leader in natural language processing",
                "deep learning is a prime example of AI"
            ]
        }
        df = pd.DataFrame(data)

        # Example word scores
        word_scores = {
            "leader": 1.0,
            "natural": 0.9,
            "language": 0.8,
            "deep": 0.7,
            "learning": 0.6
        }

        # Create the dataset
        dataset = WordContextDataset(df, word_scores, embedding_model, window_size=2)

        # Example access
        print(f"Dataset size: {len(dataset)}")
        sample = dataset[0]
        print(f"Target word: {sample['target_word']}")
        print(f"Target embedding: {sample['target_embedding']}")
        print(f"Context embedding: {sample['context_embedding']}")
        print(f"Score: {sample['score']}")
    else:
        print("Failed to create dataset due to missing embeddings.")

GloVe embeddings loaded successfully!
Dataset size: 13
Target word: the
Target embedding: [ 4.1800e-01  2.4968e-01 -4.1242e-01  1.2170e-01  3.4527e-01 -4.4457e-02
 -4.9688e-01 -1.7862e-01 -6.6023e-04 -6.5660e-01  2.7843e-01 -1.4767e-01
 -5.5677e-01  1.4658e-01 -9.5095e-03  1.1658e-02  1.0204e-01 -1.2792e-01
 -8.4430e-01 -1.2181e-01 -1.6801e-02 -3.3279e-01 -1.5520e-01 -2.3131e-01
 -1.9181e-01 -1.8823e+00 -7.6746e-01  9.9051e-02 -4.2125e-01 -1.9526e-01
  4.0071e+00 -1.8594e-01 -5.2287e-01 -3.1681e-01  5.9213e-04  7.4449e-03
  1.7778e-01 -1.5897e-01  1.2041e-02 -5.4223e-02 -2.9871e-01 -1.5749e-01
 -3.4758e-01 -4.5637e-02 -4.4251e-01  1.8785e-01  2.7849e-03 -1.8411e-01
 -1.1514e-01 -7.8581e-01]
Context embedding: [ 0.08685999  0.25556     0.09003502  0.38064998  0.61828595  0.252505
 -0.44690502  0.1213905  -0.6212535  -0.2446      0.12698    -0.678855
 -0.856985   -0.07613    -0.01766001 -0.090982    0.01950999 -0.06645
 -0.41999498  0.38361    -0.23531502  0.21082501 -0.075105   -0.05653

<hr>

### TASK 4 Build Architecture/Model
<b>Embedding Layer</b>: in simple terms, it creates word vectors of each word in the <i>word_index</i> and group words that are related or have similar meaning by analyzing other words around them.

<b>LSTM Layer</b>: to make a decision to keep or throw away data by considering the current input, previous output, and previous memory. There are some important components in LSTM.
<ul>
    <li><b>Forget Gate</b>, decides information is to be kept or thrown away</li>
    <li><b>Input Gate</b>, updates cell state by passing previous output and current input into sigmoid activation function</li>
    <li><b>Cell State</b>, calculate new cell state, it is multiplied by forget vector (drop value if multiplied by a near 0), add it with the output from input gate to update the cell state value.</li>
    <li><b>Ouput Gate</b>, decides the next hidden state and used for predictions</li>
</ul>

<b>Dense Layer</b>: compute the input with the weight matrix and bias (optional), and using an activation function. I use <b>Sigmoid</b> activation function for this work because the output is only 0 or 1.

The optimizer is <b>Adam</b> and the loss function is <b>Binary Crossentropy</b> because again the output is only 0 and 1, which is a binary number.

In [58]:
# ARCHITECTURE
EMBED_DIM = 32
LSTM_OUT = 64

model = Sequential()
model.add(Embedding(total_words, EMBED_DIM, input_length = max_length))
model.add(LSTM(LSTM_OUT))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

print(model.summary())

C:\Users\MB\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


<hr>

### Training
For training, it is simple. We only need to fit our <b>x_train</b> (input) and <b>y_train</b> (output/label) data. For this training, I use a mini-batch learning method with a <b>batch_size</b> of <i>128</i> and <i>5</i> <b>epochs</b>.

Also, I added a callback called **checkpoint** to save the model locally for every epoch if its accuracy improved from the previous epoch.

In [60]:
checkpoint = ModelCheckpoint(
    'models/LSTM.keras',
    monitor='accuracy',
    save_best_only=True,
    verbose=1
)

In [62]:
model.fit(x_train, y_train, batch_size = 128, epochs = 10, callbacks=[checkpoint])

Epoch 1/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 0.5284 - loss: 0.6845
Epoch 1: accuracy improved from -inf to 0.58938, saving model to models/LSTM.keras
313/313 ━━━━━━━━━━━━━━━━━━━━ 24s 67ms/step - accuracy: 0.5286 - loss: 0.6844
Epoch 2/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.7568 - loss: 0.5470
Epoch 2: accuracy improved from 0.58938 to 0.76050, saving model to models/LSTM.keras
313/313 ━━━━━━━━━━━━━━━━━━━━ 24s 75ms/step - accuracy: 0.7568 - loss: 0.5470
Epoch 3/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 0.6618 - loss: 0.6286
Epoch 3: accuracy did not improve from 0.76050
313/313 ━━━━━━━━━━━━━━━━━━━━ 23s 73ms/step - accuracy: 0.6616 - loss: 0.6287
Epoch 4/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - accuracy: 0.6952 - loss: 0.5980
Epoch 4: accuracy did not improve from 0.76050
313/313 ━━━━━━━━━━━━━━━━━━━━ 24s 77ms/step - accuracy: 0.6953 - loss: 0.5980
Epoch 5/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - accuracy: 0.7758 - loss: 0

<hr>

### Testing
To evaluate the model, we need to predict the sentiment using our <b>x_test</b> data and comparing the predictions with <b>y_test</b> (expected output) data. Then, we calculate the accuracy of the model by dividing numbers of correct prediction with the total data. Resulted an accuracy of <b>86.63%</b>

In [65]:
import numpy as np

# 使用 model.predict() 方法进行预测
y_pred = model.predict(x_test, batch_size=128)

# 使用 np.argmax() 函数获取预测结果的索引值
y_pred_classes = np.argmax(y_pred, axis=1)

true = 0
for i, y in enumerate(y_test):
    if y == y_pred_classes[i]:
        true += 1

print('Correct Prediction: {}'.format(true))
print('Wrong Prediction: {}'.format(len(y_pred_classes) - true))
print('Accuracy: {}'.format(true/len(y_pred_classes)*100))

79/79 ━━━━━━━━━━━━━━━━━━━━ 3s 37ms/step
Correct Prediction: 4994
Wrong Prediction: 5006
Accuracy: 49.94


---

### Load Saved Model

Load saved model and use it to predict a movie review statement's sentiment (positive or negative).

In [68]:
loaded_model = load_model('models/LSTM.keras')

Receives a review as an input to be predicted

In [73]:
review = str(input('Movie Review: '))

Movie Review:  I like cookies


The input must be pre processed before it is passed to the model to be predicted

In [75]:
# Pre-process input
regex = re.compile(r'[^a-zA-Z\s]')
review = regex.sub('', review)
print('Cleaned: ', review)

words = review.split(' ')
filtered = [w for w in words if w not in english_stops]
filtered = ' '.join(filtered)
filtered = [filtered.lower()]

print('Filtered: ', filtered)

Cleaned:  I like cookies
Filtered:  ['i like cookies']


Once again, we need to tokenize and encode the words. I use the tokenizer which was previously declared because we want to encode the words based on words that are known by the model.

In [78]:
tokenize_words = token.texts_to_sequences(filtered)
tokenize_words = pad_sequences(tokenize_words, maxlen=max_length, padding='post', truncating='post')
print(tokenize_words)

[[    1     6 16274     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0]]


This is the result of the prediction which shows the **confidence score** of the review statement.

In [81]:
result = loaded_model.predict(tokenize_words)
print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 204ms/step
[[0.9371112]]


If the confidence score is close to 0, then the statement is **negative**. On the other hand, if the confidence score is close to 1, then the statement is **positive**. I use a threshold of **0.7** to determine which confidence score is positive and negative, so if it is equal or greater than 0.7, it is **positive** and if it is less than 0.7, it is **negative**

In [84]:
if result >= 0.7:
    print('positive')
else:
    print('negative')

positive


#### Difficulties Encountered
- One of the main difficulties was the technical issue with TorchText and Python version compatibility. This required a shift in strategy to use Kaggle and Pandas for data loading, which was not the initial plan.
- Problem of datas that had different shape
- Problem related to keras : in the function to fit the model, we are using sequential model and it says sequential has no outputs yet. Therefore, we tried to look for the issue on stackoverflow and github but we did not get response so we continued to debug it.

#### Steps taken to alleviate difficulties
To address the TorchText and Python version compatibility issue, we opted to use Kaggle to directly download the dataset and then import it into Pandas for preprocessing. This change in strategy allowed us to bypass the initial dependency from TorchText. For the challenge of data with varying shapes, I implemented data preprocessing techniques such as padding and truncation to normalize the input data, ensuring that all sequences were of equal length and suitable for model training. The website stack helped also while we were facing errors.

#### General description of what you did, explain how you understood the task and what you did to solve it in general language, no code.
For this task, we built a model that could classify the sentiment of individual words based on the sentiment of the sentences they appear in. To do so, we first prepared the data by tokenizing the text into words and sentences, and then padded the sentences to a uniform length to standardize the input for the model. Then, we developed a neural network model designed to process sequences of words and learn the contextual relationships that influence word sentiment. The model was trained on the IMDb dataset, using the sentence-level sentiment labels to teach it to classify words as positive, negative, or neutral. After training, the model was evaluated to assess its ability to accurately classify word sentiments in new sentences.

#### Potential limitations of our approach, what could be issues, how could this be hard on different data or with slightly different conditions
- **Data Bias**: The IMDb dataset may contain biases that could affect the model's performance on different types of data or under slightly different conditions.
- **Context Length**: The model might struggle with longer contexts or sentences where the relationship between words and sentiment is more complex.
- **Vocabulary Size**: Limitations in the model's vocabulary size could lead to misclassifications of words that are not well-represented in the training data.

#### Extensions and Future Work
An interesting extension of this work could involve exploring different neural network architectures, such as transformers, which are known for their ability to handle long-range dependencies in text. Additionally, incorporating multimodal data, such as movie scenes or audio, could provide richer context for sentiment analysis. 
- This approach could also be extended to detect hate speech or discriminatory comments on social media platforms more effectively.
- For hotel, restaurant and product that base their grade(with stars usuall) on the positive review.
